In [1]:
import requests
import urllib.error
import urllib.parse
from urllib import request
import json
import pandas as pd
import numpy as np
import re
import time
from datetime import date, datetime

In [2]:
dataframe = pd.read_excel('table of events and activities.xlsx')

In [3]:
dataframe.head(3)

,Activity,Day,Category,Address,Contact,Website,Email,Description
0,Carer Support Group,Contact organiser,"Social Groups, Health and Wellbeing",Kathleen Syme Library and Community Centre 251...,9658 9989,https://www.melbourne.vic.gov.au/olderpeople,healthyageing@melbourne.vic.gov.au,Anyone can become a carer at any stage of thei...
1,Carlton Baths,Daily,Health and Wellbeing,"248 Rathdowne Street, Carlton",9347 3677,https://www.carltonbaths.ymca.org.au,carltonbaths@ymca.org.au,Carlton Baths welcomes members and visitors to...
2,Carlton Farmers Market,First Saturday of the month,Health and Wellbeing,Carlton North Primary School. Corner Canning a...,"9499 3400, 0429 146 627",https://www.mfm.com.au,NaN,"Buy your meat, fruit and vegetables as well as..."


In [4]:
dataframe['geocode address'] = ''

In [5]:
for i in range(len(dataframe)):
    try:
        dataframe['geocode address'][i] = dataframe['Address'][i] + ', Melbourne'
    except TypeError:
        print(i)

In [6]:
def read_the_url(urllink):
    res = requests.get(urllink)
    js = json.loads(res.text)
    return js



def get_latitude_longtitude(address):
    # decode url
    address = urllib.parse.quote(address)
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + address + "&key=" #add your key    
    while True:
        res = requests.get(url)
        js = json.loads(res.text)

        if js["status"] != "OVER_QUERY_LIMIT":
            time.sleep(1)
            break

    result = js["results"][0]["geometry"]["location"]
    lat = result["lat"]
    lng = result["lng"]

    return lat, lng

In [7]:
dataframe['lat'] = ''
dataframe['lng'] = ''

In [8]:
for i in range(len(dataframe)):
    address = dataframe['geocode address'][i]
    lat, lng = get_latitude_longtitude(address)
    dataframe['lat'][i] = lat
    dataframe['lng'][i] = lng

In [9]:
dataframe.head()

,Activity,Day,Category,Address,Contact,Website,Email,Description,geocode address,lat,lng
0,Carer Support Group,Contact organiser,"Social Groups, Health and Wellbeing",Kathleen Syme Library and Community Centre 251...,9658 9989,https://www.melbourne.vic.gov.au/olderpeople,healthyageing@melbourne.vic.gov.au,Anyone can become a carer at any stage of thei...,Kathleen Syme Library and Community Centre 251...,-37.7987,144.965
1,Carlton Baths,Daily,Health and Wellbeing,"248 Rathdowne Street, Carlton",9347 3677,https://www.carltonbaths.ymca.org.au,carltonbaths@ymca.org.au,Carlton Baths welcomes members and visitors to...,"248 Rathdowne Street, Carlton, Melbourne",-37.7935,144.972
2,Carlton Farmers Market,First Saturday of the month,Health and Wellbeing,Carlton North Primary School. Corner Canning a...,"9499 3400, 0429 146 627",https://www.mfm.com.au,NaN,"Buy your meat, fruit and vegetables as well as...",Carlton North Primary School. Corner Canning a...,-37.7911,144.973
3,Carlton Neighbourhood Learning Centre,Daily,"Arts and culture, Social Groups, Health and We...","20 Princes Street, Carlton",9347 2739,https://www.cnlc.org.au,info@cnlc.org.au,Take part in a broad and varied program that i...,"20 Princes Street, Carlton, Melbourne",-37.7927,144.975
4,Carlton Women’s Turkish Group (cohealth),Tuesday,Social Groups,"510 Lygon Street, Carlton",9448 6792,https://www.cohealth.org.au,maria.ibrahim@cohealth.org.au,Connect and share experiences with other Turki...,"510 Lygon Street, Carlton, Melbourne",-37.7946,144.968


In [10]:
class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        else:
            return json.JSONEncoder.default(self, obj)

In [11]:
dataframe.drop(columns=['geocode address'], inplace=True)

In [14]:
for i in range(len(dataframe)):
    list_of_dict = []
    if type(dataframe['Category'][i]) == str:
        list_of_category = dataframe['Category'][i].split(',')
        for item in list_of_category:
            dict_of_category = {'category':item}
            list_of_dict.append(dict_of_category)
    dataframe['Category'][i] = list_of_dict

In [15]:
jsonlist = []
for i in range(len(dataframe)):
    aItem = {}
    aItem["id"] = i
    aItem["Activity"] = dataframe['Activity'][i]
    aItem['OpenTime'] = dataframe['Day'][i]
    aItem["Category"] = dataframe['Category'][i]
    aItem['Address'] = dataframe['Address'][i]
    aItem['Phone'] = dataframe['Contact'][i]
    aItem['Website'] = dataframe['Website'][i]
    aItem['Email'] = dataframe['Email'][i]
    aItem['desc'] = dataframe['Description'][i]
    aItem['lat'] = dataframe['lat'][i]
    aItem['lng'] = dataframe['lng'][i]
    aJson = json.dumps(aItem, cls=ComplexEncoder)
    jsonlist.append(aItem)

In [16]:
savefile = 'well-being activities.json'

with open(savefile, 'w') as f:
    f.write(json.dumps(jsonlist, cls=ComplexEncoder))